In [61]:
import numpy as np
import pandas as pd
import sys 

# PROBLEMA 1:

#------------------------------------------------
def f(t,u):
    f = u + 2*np.exp(-t)
    return f
#------------------------------------------------
def z(wi,ti,t):
    z = (wi*np.exp(-ti) + np.exp(-2*ti) - np.exp(-2*t))*np.exp(t)
    return z
#------------------------------------------------
def u(t): #Solución
    u = (-1/np.exp(t)) + 2*np.exp(t)
    return u
#------------------------------------------------
def TE(f,u,t0,u0,imax,h): #Trapeció Explícito
    w = np.zeros(imax+1)
    t = np.zeros(imax+1)
    g = np.zeros(imax+1) #Error global
    e = np.zeros(imax+1) #Error local
    w[0] = u0
    t[0] = t0
    for i in range(0,imax):
        t[i+1] = t[i] + h
        w[i+1] = w[i] + 0.5*h*(f(t[i],w[i]) + f(t[i]+h, w[i] + h*f(t[i],w[i])))
        e[i+1] = abs(z(w[i],t[i],t[i+1]) - w[i+1])
    g = abs(u(t[imax-1])- w[imax-1])
    return (t,w,g,e)
#------------------------------------------------
def erc(E1,E2,h1,h2):
    q = np.log(E1/E2)/np.log(h1/h2)
    return q
#------------------------------------------------
def LargestLocalErrorTE(f,u,t0,u0,imax,h):
    lle = max(TE(f,u,t0,u0,imax,h)[3])
    return lle
#------------------------------------------------
print("PROBLEMA 1: \t\t")
print("Sucesion t_i:", TE(f,u,0.0,1.0,10,0.1)[0])
print("Sucesion w_i:", TE(f,u,0.0,1.0,10,0.1)[1])
print("Sucesion de error global g_i:", TE(f,u,0.0,1.0,10,0.1)[2])
print("Error global en t = 1",TE(f,u,0.0,1.0,10,0.1)[2])
print("Sucesion de error local e_i:", TE(f,u,0.0,1.0,10,0.1)[3])
print("Maximo de los errores locales:", LargestLocalErrorTE(f,u,0.0,1.0,10,0.1))
#------------------------------------------------
registros1 = []

for i in range(0,6):
    n = 10*(2**i)
    k = 2**(-i)
    h = (0.1)*k
    lle = LargestLocalErrorTE(f,u,0.0,1.0,n,h)
    glob = TE(f,u,0.0,1.0,n,h)[2]
    registro = (n,h,lle,glob)
    registros1.append(registro)
    
errores1 = []

for i in range(0,6):
    if i == 0:
        erce = None
        ercg = None
    else:
        erce = erc(registros1[i-1][2],registros1[i][2],registros1[i-1][1],registros1[i][1])
        ercg = erc(registros1[i-1][3],registros1[i][3],registros1[i-1][1],registros1[i][1])
    errores1.append((erce,ercg))
    
df1 = pd.DataFrame(registros1,columns = ["n","h","largest local error", "global error in T = 1"])
df2 = pd.DataFrame(errores1,columns = ["erc(e)","erc(g)"])
dfP1 = pd.concat((df1,df2),axis = 1)
dfP1 = dfP1.reindex(columns=["n","h","largest local error","erc(e)","global error in T = 1","erc(g)"])
print(dfP1)
dfP1.to_excel("Tabla Problema 1.xlsx")


# PROBLEMA 2:

def RK4(f,u,t0,u0,imax,h):
    w = np.zeros(imax+1)
    w[0] = u0
    t = np.zeros(imax+1) 
    t[0] = t0
    e = np.zeros(imax+1)
    g = 0
    for i in range(0,imax):
        s1 = f(t[i], w[i])
        s2 = f(t[i]+h/2, w[i]+(h/2)*s1)
        s3 = f(t[i]+h/2, w[i]+(h/2)*s2)
        s4 = f(t[i]+h, w[i]+h*s3)
        w[i+1] = w[i]+(h/6)*(s1+2*s2+2*s3+s4)
        t[i+1] = t[i]+h
        e[i] = abs(z(w[i],t[i],t[i+1]) - w[i+1])
    g = abs(u(t[imax])-w[imax])
    return (t,w,g,e)
# ----------------------------------------------------------
def LargestLocalErrorRK4(f,u,t0,u0,imax,h):
    lle = max(RK4(f,u,t0,u0,imax,h)[3])
    return lle 

# Calcular el épsilon de la máquina

sys.float_info.epsilon
eps = 1.0
while eps + 1 > 1:
    eps /= 2
eps *=2
k = 1000 # pueden ser otros valores 
h = 1/k

def f1(t, u1):
    return 1+7*t

def u1(t):
    return (7/2)*(t**2) + t +1

def f2(t, u2):
    return t**2 -t

def u2(t):
    return (1/6)*(2*(t**3)-3*(t**2)+6)

def f3(t, u3):
    return 2*(t**3)-2*(t**2)

def u3(t):
    return (1/6)*(3*(t**4)-4*(t**3)+6)


def f4(t, u4):
    return (t**4)-(t**3)-t

def u4(t):
    return ((t**5)/5)-((t**4)/4)-((t**2)/2)+1

def f5(t, u5):
    return t**5

def u5(t):
    return (1/6)*((t**6)+6)


P1RK4 = RK4(f1,u1,0,1, k, h)
P2RK4 = RK4(f2,u2, 0, 1, k, h)
P3RK4 = RK4(f3,u3,0,1,k,h)
P4RK4 = RK4(f4,u4,0,1,k,h)
P5RK4 = RK4(f5,u5,0,1,k,h)
print("\t\tPROBLEMA 2: \t\t")
print("Epsilon de la maquina en valor absoluto:", abs(eps))
print("Error global después de " , k, " iteraciones para un polinomio de grado 1: ", P1RK4[2])
print("Error global después de " , k, " iteraciones para un polinomio de grado 2: ", P2RK4[2])
print("Error global después de " , k, " iteraciones para un polinomio de grado 3: ", P3RK4[2])
print("Error global después de " , k, " iteraciones para un polinomio de grado 4: ", P4RK4[2])
print("Error global después de " , k, " iteraciones para un polinomio de grado 5: ", P5RK4[2])

registros2 = []
for i in range(1,4):
    n = 10*pow(2,i)
    h = 1/n
    lle = LargestLocalErrorRK4(f,u,0,1,n,h)
    glob = RK4(f,u,0,1,n,h)[2]
    registro = (n,h,lle,glob)
    registros2.append(registro)
errores2 = []
for i in range(0,3):
    if i == 0:
        erce = None 
        ercg = None 
    else:
        erce = erc(registros2[i-1][2],registros2[i][2],registros2[i-1][1],registros2[i][1])
        ercg = erc(registros2[i-1][3],registros2[i][3],registros2[i-1][1],registros2[i][1])
    errores2.append((erce,ercg))

df3 = pd.DataFrame(registros2,columns = ["n","h","largest local error", "global error in T = 1"])
df4 = pd.DataFrame(errores2,columns = ["erc(e)","erc(g)"])
dfP2 = pd.concat((df3,df4),axis = 1)
dfP2 = dfP2.reindex(columns=["n","h","largest local error","erc(e)","global error in T = 1","erc(g)"])
print(dfP2)
dfP2.to_excel("Tabla Problema 2.xlsx")


# Problema 3:
    
print("PROBLEMA 3: \t\t")

# Trapecio Explicito con h1 = 1/10 y error global en t = 1

print("Error global del PVI (1) en t = 1 para el Trapecio Explicito con h1 = 1/10:", TE(f,u,0,1,10,0.1)[2])

#Trapecio Explicito con h2 = 1/20 y error global en t = 1

print("Error global del PVI (1) en t = 1 para el Trapecio Explicito con h2 = h1/2", TE(f,u,0,1,20,0.05)[2])

#RK4 con h1 = 1/10 y error global en t = 1

print("Error global del PVI (1) en t = 1 para el metodo RK4 con h1 = 1/10: ", RK4(f,u,0,1,10,0.1)[2])

errores3 = (TE(f,u,0,1,10,0.1)[2],TE(f,u,0,1,20,0.05)[2],RK4(f,u,0,1,10,0.1)[2])
evaluaciones = (10,20,10)

dfP3 = pd.DataFrame((errores3,evaluaciones),index = ["Error global en t = 1","# de evaluaciones de f"],columns = ["T.E. (h1)","T.E. (h2)","R.K.4 (h1)"])
print(dfP3)
dfP3.to_excel("Tabla Problema 3.xlsx")

PROBLEMA 1: 		
Sucesion t_i: [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
Sucesion w_i: [1.         1.30548374 1.62396473 1.95862323 2.31280067 2.69003302
 3.09408602 3.52899286 3.99909439 4.50908246 5.06404672]
Sucesion de error global g_i: 0.003554106900959475
Error global en t = 1 0.003554106900959475
Sucesion de error local e_i: [0.00000000e+00 2.06763117e-05 8.71863992e-05 1.54557373e-04
 2.23462277e-04 2.94589375e-04 3.68649030e-04 4.46380800e-04
 5.28560817e-04 6.16009542e-04 7.09599953e-04]
Maximo de los errores locales: 0.0007095999534287856
     n         h  largest local error    erc(e)  global error in T = 1  \
0   10  0.100000         7.096000e-04       NaN               0.003554   
1   20  0.050000         9.325952e-05  2.927683               0.001073   
2   40  0.025000         1.194788e-05  2.964497               0.000294   
3   80  0.012500         1.511797e-06  2.982418               0.000077   
4  160  0.006250         1.901239e-07  2.991253               0.000020  